# Model Evaluation

In [1]:
import numpy as np
import shap
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# 1. Core Performance Metrics
def calculate_metrics(y_true, y_pred):
    return {
        'MAE': mean_absolute_error(y_true, y_pred),
        'RMSE': np.sqrt(mean_squared_error(y_true, y_pred)),
        'R²': r2_score(y_true, y_pred),
        'Error Std': np.std(y_true - y_pred),
        'Max Error': np.max(np.abs(y_true - y_pred))
    }

baseline_metrics = calculate_metrics(y_test, model.predict(X_test))
optimized_metrics = calculate_metrics(y_test, optimized_model.predict(X_test))

print("Baseline Model:")
print(baseline_metrics)
print("\nOptimized Model:")
print(optimized_metrics)

# 2. Clinical Impact Analysis
def clinical_error_analysis(y_true, y_pred):
    df = pd.DataFrame({'true': y_true, 'pred': y_pred})
    
    # Critical thresholds (GEMA guidelines)
    df['true_risk'] = np.where(df['true'] > 2, 'High Risk', 'Normal')
    df['pred_risk'] = np.where(df['pred'] > 2, 'High Risk', 'Normal')
    
    # Confusion matrix
    confusion = pd.crosstab(df['true_risk'], df['pred_risk'])
    
    # Clinical metrics
    sensitivity = confusion.loc['High Risk', 'High Risk']/confusion.loc['High Risk'].sum()
    specificity = confusion.loc['Normal', 'Normal']/confusion.loc['Normal'].sum()
    
    return {
        'confusion_matrix': confusion,
        'sensitivity': sensitivity,
        'specificity': specificity,
        'clinical_accuracy': (confusion.values.diagonal().sum()/confusion.values.sum())
    }

clinical_results = clinical_error_analysis(y_test, optimized_model.predict(X_test))

plt.figure(figsize=(10,6))
sns.heatmap(clinical_results['confusion_matrix'], annot=True, fmt='d', cmap='Blues')
plt.title('Clinical Risk Classification Performance\n(Threshold: 2 puffs/hour)')
plt.savefig('clinical_confusion.png', dpi=300, bbox_inches='tight')
plt.show()

# 3. SHAP Analysis for Clinical Interpretability
explainer = shap.Explainer(optimized_model)
shap_values = explainer(X_train)

plt.figure(figsize=(12,6))
shap.summary_plot(shap_values, X_train, plot_type='bar', show=False)
plt.title('Feature Impact on Inhaler Usage Predictions')
plt.savefig('shap_summary.png', dpi=300, bbox_inches='tight')
plt.close()

# 4. Temporal Error Analysis
def plot_temporal_errors(df, time_col='hour'):
    df['error'] = np.abs(df['true'] - df['pred'])
    
    plt.figure(figsize=(14,6))
    
    # Pollution vs Error
    plt.subplot(1,2,1)
    sns.scatterplot(x='pm25_24h_avg', y='error', data=df,
                   hue='gema_risk_score', palette='viridis')
    plt.axvline(25, color='r', linestyle='--', label='WHO PM2.5 Limit')
    plt.title('Prediction Error vs PM2.5 Exposure')
    plt.xlabel('24h Average PM2.5 (µg/m³)')
    plt.ylabel('Absolute Error (Puffs)')
    
    # Time-of-Day Pattern
    plt.subplot(1,2,2)
    sns.boxplot(x=time_col, y='error', data=df,
               showfliers=False, palette='coolwarm')
    plt.title('Error Distribution by Hour of Day')
    plt.xlabel('Hour of Day')
    plt.ylabel('Absolute Error (Puffs)')
    
    plt.tight_layout()
    plt.savefig('temporal_errors.png', dpi=300)
    plt.show()

error_df = X_test.copy()
error_df['true'] = y_test.values
error_df['pred'] = optimized_model.predict(X_test)
plot_temporal_errors(error_df)

# 5. Spatial Error Analysis
def plot_spatial_errors(df):
    district_errors = df.groupby('district').agg({
        'error': 'mean',
        'pm25_24h_avg': 'mean',
        'gema_risk_score': 'mean'
    }).sort_values('error', ascending=False)
    
    plt.figure(figsize=(12,6))
    sns.heatmap(district_errors.T, annot=True, fmt=".1f",
               cmap="YlOrRd", linewidths=.5)
    plt.title('Spatial Error Analysis by District')
    plt.xlabel('District')
    plt.ylabel('Metrics')
    plt.savefig('spatial_errors.png', dpi=300)
    plt.show()

plot_spatial_errors(error_df)

# 6. Business Impact Projection
def calculate_business_impact(y_true, y_pred):
    # Preventable exacerbations (assuming 10 puffs/day = 1 exacerbation)
    preventable = np.sum((y_true >= 10) & (y_pred >= 8)) / np.sum(y_true >= 10)
    
    # Cost savings (simulated)
    hospital_cost = 5000  # USD per exacerbation
    daily_patients = 1000
    savings = preventable * daily_patients * hospital_cost * 0.3  # 30% prevention
    
    return {
        'preventable_exacerbations': f"{preventable*100:.1f}%",
        'estimated_daily_savings': f"${savings:,.0f}"
    }

impact = calculate_business_impact(y_test, optimized_model.predict(X_test))
print("\nBusiness Impact:")
print(impact)

/home/ivanseldasp/anaconda3/envs/asthma-realtime-py39/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


NameError: name 'y_test' is not defined